In [1]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')


In [9]:
#---------------------------------------------------------------------------------------------------------------
#- Exercici 2
#Repeteix l'exercici 1 amb el dataset que disposem en el repositori de GitHub PRE-PROCESSING-DATA, movies.dat

#la serpació en aquest cas són els ::
df = pd.read_csv('movies.dat', sep='::',names=["index","nom","tipus"])
print(df.dtypes)
#canvio el camp tipus,de string a una llista treient el seperador |
df['tipus'] = df.tipus.apply(lambda x: x.split('|'))
print(df.dtypes)

#per cada valor de la llistat de tipus, desplaguem en files
tipus = df['tipus'].apply(pd.Series).reset_index().melt(id_vars='index').dropna()[['index', 'value']].set_index('index')
#fem un merge per obtenir la taula original, i cada valor de tipus en una fila

total=pd.merge(
    tipus,
    df[["nom"]],
    left_index=True,
    right_index=True)


#treiem l'any del nom per poder-ho treballar estadísticament
total['any1'] = total.nom.str.extract('(\(\d\d\d\d\))',expand=False)
total['any1'] = total.any1.str.extract('(\d\d\d\d)',expand=False)                                         
                                            

#pivoto la taula perquè cada caracteristica sigui una columna a fi de poder treballar-hi estadísticament
#utilitzo el size, perque no tinguin en compte si hi haguessin nuls de les dades
result = total.groupby(['nom', 'any1', 'value']).size().reset_index().pivot_table(total, index = ['nom', 'any1'], columns="value", aggfunc = 'count').reset_index().fillna(0)

#treiem totals de tipus de pelicules per any
result1 = result.groupby(['any1']).sum()
#.reset_index().pivot_table(total, index = ['any1'], columns="value", aggfunc = 'count').reset_index().fillna(0)

print(total.head())
print(result.head())
print(result1.head())

index     int64
nom      object
tipus    object
dtype: object
index     int64
nom      object
tipus    object
dtype: object
        value               nom  any1
0   Animation  Toy Story (1995)  1995
0  Children's  Toy Story (1995)  1995
0      Comedy  Toy Story (1995)  1995
1   Adventure    Jumanji (1995)  1995
1  Children's    Jumanji (1995)  1995
                                 nom  any1      0                      \
value                                      Action Adventure Animation   
0             $1,000,000 Duck (1971)  1971    0.0       0.0       0.0   
1               'Night Mother (1986)  1986    0.0       0.0       0.0   
2          'Til There Was You (1997)  1997    0.0       0.0       0.0   
3                 'burbs, The (1989)  1989    0.0       0.0       0.0   
4      ...And Justice for All (1979)  1979    0.0       0.0       0.0   

                                                                          \
value Children's Comedy Crime Documentary Drama Fantasy Film

In [3]:
result1.shape

(81, 18)

In [4]:
result1.columns

MultiIndex([('counts',      'Action'),
            ('counts',   'Adventure'),
            ('counts',   'Animation'),
            ('counts',  'Children's'),
            ('counts',      'Comedy'),
            ('counts',       'Crime'),
            ('counts', 'Documentary'),
            ('counts',       'Drama'),
            ('counts',     'Fantasy'),
            ('counts',   'Film-Noir'),
            ('counts',      'Horror'),
            ('counts',     'Musical'),
            ('counts',     'Mystery'),
            ('counts',     'Romance'),
            ('counts',      'Sci-Fi'),
            ('counts',    'Thriller'),
            ('counts',         'War'),
            ('counts',     'Western')],
           names=[None, 'value'])

In [5]:
result1.describe().round(3)

counts                                                              \
value  Action Adventure Animation Children's   Comedy   Crime Documentary   
count  81.000    81.000    81.000     81.000   81.000  81.000      81.000   
mean    6.210     3.494     1.296      3.099   14.815   2.605       1.568   
std    10.819     5.292     2.046      5.083   27.465   5.328       4.407   
min     0.000     0.000     0.000      0.000    0.000   0.000       0.000   
25%     0.000     0.000     0.000      0.000    2.000   0.000       0.000   
50%     1.000     2.000     0.000      1.000    3.000   1.000       0.000   
75%     6.000     4.000     2.000      3.000   10.000   2.000       1.000   
max    45.000    25.000     8.000     22.000  115.000  26.000      22.000   

                                                                          \
value    Drama Fantasy Film-Noir  Horror Musical Mystery Romance  Sci-Fi   
count   81.000  81.000    81.000  81.000  81.000  81.000  81.000  81.000   
mean    19.790   0.840     0.543   4.235   1.407   1.309   5.815   3.407   
std     37.677   1.495     1.001   4.996   1.403   2.432  12.433   4.587   
min      0.000   0.000     0.000   0.000   0.000   0.000   0.000   0.000   
25%      4.000   0.000     0.000   0.000   0.000   0.000   0.000   0.000   
50%      6.000   0.000     0.000   2.000   1.000   1.000   2.000   1.000   
75%     12.000   1.000     1.000   7.000   2.000   1.000   4.000   5.000   
max    166.000   6.000     4.000  21.000   7.000  15.000  58.000  18.000   

                                
value Thriller     War Western  
count   81.000  81.000  81.000  
mean     6.074   1.765   0.840  
std     11.852   2.271   1.209  
min      0.000   0.000   0.000  
25%      1.000   0.000   0.000  
50%      2.000   1.000   0.000  
75%      4.000   2.000   1.000  
max     55.000  12.000   6.000